In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Suppose you want to predict a continuous output (regression) from a stack of continuous inputs.  In this example, the output is impervious surface area from [NLCD](https://www.mrlc.gov/data) and the input is a Landsat 8 composite.  The model is a [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597). This notebook shows:

1.   Exporting training/testing patches from Earth Engine, suitable for training an FCNN model.
2.   Preprocessing.
3.   Training and validating an FCNN model.
4.   Making predictions with the trained model and importing them to Earth Engine.

# Initial package

Declare the variables that will be in use throughout the notebook.

In [1]:
PACKAGE_PATH = 'Water_classification_package'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 12
drwxr-xr-x 1 root root 4096 Jul 13 13:43 sample_data
drwxr-xr-x 3 root root 4096 Jul 23 09:28 wandb
drwxr-xr-x 3 root root 4096 Jul 23 09:27 Water_classification_package
mkdir: cannot create directory ‘Water_classification_package’: File exists
total 20
-rw-r--r-- 1 root root 1666 Jul 23 09:26 config.py
-rw-r--r-- 1 root root    0 Jul 23 09:29 __init__.py
-rw-r--r-- 1 root root 7985 Jul 23 09:26 metrics_.py
-rw-r--r-- 1 root root 3201 Jul 23 09:26 preprocessing.py
drwxr-xr-x 2 root root 4096 Jul 23 09:27 __pycache__


In [2]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

CONFIG = None

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

# Acc = TP + TN / (TP + TN + FP + FN)
# possible_pos = TP + FN
# predicted_pos = TP + FP
# Missing TN
# TN = total - possible_pos - predicted_pos + TP
# TP + TN + FP + FN = possible_pos + predicted_pos - TP + TN

def custom_accuracy(y_true, y_pred):
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
  
    # total_data = tf.cast(total_data, tf.float32|tf.int32)
    # true_positives = tf.cast(true_positives, tf.float32|tf.int32)
    # possible_positives = tf.cast(possible_positives, tf.float32|tf.int32)
    # predicted_positives = tf.cast(predicted_positives, tf.float32|tf.int32)
    # print(K.int_shape(y_true), K.int_shape(y_pred))
    # print(K.int_shape(y_pred)[0], K.int_shape(y_pred)[1], K.int_shape(y_pred)[2])
    # print(total_data)
    # print(possible_positives)
    # (true_positives) / (total_data + K.epsilon())
    return (true_positives + true_negatives) / (total_data + K.epsilon())



def MetricCalculator_backup(model, test_data, total_steps):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  # total_steps = 2000
  test_acc_metric = tf.keras.metrics.Accuracy()
  # test_F1_metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    # print(y_pred[0][1] == 1, y_pred[0][1] == 1)
    for j in range(y_pred.shape[0]):
      if y_true[j][1] == 1 and y_pred[j][1] == 1:
        TP += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 0:
        TN += 1
      if y_true[j][1] == 1 and y_pred[j][1] == 0:
        FN += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 1:
        FP += 1
    test_acc_metric.update_state(y_true, y_pred)
  print("TP: ", TP)
  print("TN: ", TN)
  print("FP: ", FP)
  print("FN: ", FN)


  if TP != 0:
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    F1 = 2*(recall*precision)/(recall + precision)
  else:
    precision = 0
    recall = 0
    F1 = -1

  print("precision: ", precision)
  print("recall: ", recall)
  print("F1_Score: : ", F1)
  print("Accuracy: ", test_acc_metric.result().numpy())
  return precision, recall, F1, test_acc_metric.result().numpy()

def MetricCalculator(model, test_data, total_steps):
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy



def MetricCalculator_multiview_2(model, test_data, total_steps):
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

def MetricCalculator_multiview_3(model, test_data, total_steps):
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2, x3 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2),len(CONFIG.BANDS3)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2, x3]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

Overwriting Water_classification_package/metrics_.py


In [3]:
%%writefile {PACKAGE_PATH}/config.py
import tensorflow as tf
from . import metrics_
class configuration:
  def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE, BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None, type_=1):
    if type_ == 1:
      self.type_ = "fs"
    elif type_ == 2:
      self.type_ = "m2"
    elif type_ == 3:
      self.type_ = "m3"
    else:
      self.type_ = None
    self.country = country
    self.PROJECT_TITLE = PROJECT_TITLE
    self.BANDS1 = BANDS1
    self.BANDS2 = BANDS2
    self.BANDS3 = BANDS3
    self.BUCKET = "geebucketwater"
    self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
    self.TRAIN_SIZE = TRAIN_SIZE
    self.EVAL_SIZE = EVAL_SIZE
    self.BUCKET = "geebucketwater"
    self.TRAINING_BASE = f'training_patches'
    self.EVAL_BASE = f'eval_patches'
    self.TEST_BASE_1 = f'test_patches_1'
    self.TEST_BASE_2 = f'test_patches_2'
    self.RESPONSE = 'water'
    self.BANDS = BANDS1 + BANDS2 + BANDS3 
    self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
    # Specify the size and shape of patches expected by the model.
    self.KERNEL_SIZE = 256
    self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
    self.COLUMNS = [
      tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32) for k in self.FEATURES
    ]
    self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
    # Specify model training parameters.
    self.BATCH_SIZE = 16
    self.EPOCHS = 5
    self.BUFFER_SIZE = 2000
    self.OPTIMIZER = 'adam'
    self.LOSS = 'categorical_crossentropy'
    self.METRICS = ['AUC', "categorical_accuracy", metrics_.f1]
    self.image = image
    self.sam_arr = sam_arr
    




Overwriting Water_classification_package/config.py


In [4]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf

class Preprocessor:
  def __init__(self, config):
    self.config = config

  def parse_tfrecord(self, example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
      example_proto: a serialized Example.
    Returns:
      A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, self.config.FEATURES_DICT)


  def to_tuple(self, inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
      inputs: A dictionary of tensors, keyed by feature name.
    Returns:
      A tuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in self.config.FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(self.config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(self.config.BANDS):], tf.int32), depth=2),[256,256,2])


  def get_dataset(self, pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
      pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
      A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
    return dataset

  def get_training_dataset(self, location):
    """Get the preprocessed training dataset
    Returns: 
      A tf.data.Dataset of training data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.shuffle(self.config.BUFFER_SIZE).batch(self.config.BATCH_SIZE).repeat()
    return dataset

  def get_training_dataset_for_testing(self, location):
    """Get the preprocessed training dataset
    Returns: 
      A tf.data.Dataset of training data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

  def get_eval_dataset(self, location):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "eval_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

  # print(iter(evaluation.take(1)).next())

  def get_test_dataset(self, location, test_base):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + test_base + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset



Overwriting Water_classification_package/preprocessing.py


# Setup software libraries

Authenticate and import as necessary.

In [5]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()


project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=gdMj7iPfMfeSuLHnm1N8uYJ83amWRdXRVcqgjBP51SY&tc=OfJfyN9CE-cotXeO5U0mzA0yDMB3ouUsuYe4VK3EoGs&cc=rZdNfMM7DTdM_ytFxVPkASru3zsjie6r7RyHLBT-kMs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgl03-7QaKD9-Mm7vLNxm9PSX1y34IOJLJe2k6FQ6be92DjsBFI8iU

Successfully saved authorization token.
Updated property [core/project].
2.8.2


In [6]:
from importlib import reload
# reload(preprocessing)
from Water_classification_package import config, metrics_, preprocessing

In [7]:
configs_fs_global = {}
train_size = 72*10
eval_size = 72*3

#### Feature stack experiment

configs_fs_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, country = "global")
configs_fs_global["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size, country = "global")

###### Multiexperiment

configs_multi_global = {}

configs_multi_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")


configs_multi_global["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2, country = "global")
configs_multi_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2, country = "global")

configs_multi_3_global = {}

configs_multi_3_global["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3, country = "global")

configs_multi_3_global["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [8]:
%%capture
!pip install wandb --upgrade

In [9]:
import wandb
from wandb.keras import WandbCallback
import time
import pandas as pd

wandb.init(project='kl-121-dissertation', reinit=True)

df_train_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_eval_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_train_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_eval_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])

df_test_everywhere_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_everywhere_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_everywhere_3 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])

df_test1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test3 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test4 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test5 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test6 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test7 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test8 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test9 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test10 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m2_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m2_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m3_1 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])
df_test_m3_2 = pd.DataFrame(columns=['Name', 'precision', 'recall', "F1", "accuracy"])

wandb: Currently logged in as: kl-121 (kl-121-dissertation). Use `wandb login --relogin` to force relogin


# FeatureStack

## Training and Evaluation

In [ ]:
for i in range(0, len(list(configs_fs_global))):
  conf = configs_fs_global[list(configs_fs_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  wandb.run.name = "metrics_train_eval_feature_stack_tibet_global"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})
  training = preproc.get_training_dataset_for_testing("train_in_global/")
  evaluation = preproc.get_eval_dataset("train_in_global/")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator(model_custom, training, 72*10)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator(model_custom, evaluation, 72*3)
  df_train_1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_eval_1.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  wandb.log({'df_featurestack_train_global': df_train_1})
  wandb.log({'df_featurestack_eval_global': df_eval_1})

gs://geebucketwater/fs_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9435193352023097
recall_macro:  0.7954235942492809
F1_macro_Score: :  0.8533525174786292
Accuracy:  0.9809348000420465


  0%|          | 0/216 [00:00<?, ?it/s]

precision_macro:  0.7088341047750414
recall_macro:  0.5458239275586327
F1_macro_Score: :  0.5725314718632458
Accuracy:  0.9816065187807437
gs://geebucketwater/fs_global_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


...

precision_macro:  0.49240954233718043
recall_macro:  0.49945082860840023
F1_macro_Score: :  0.4953475088583995
Accuracy:  0.9808523390028212
gs://geebucketwater/fs_global_Cnn_L8SR_S1A/Models/L8SR_S1A_EPOCHS_10
L8SR_S1A


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9693558644799376
recall_macro:  0.8931917500409823
F1_macro_Score: :  0.9276089236426397
Accuracy:  0.9896312713623047


  0%|          | 0/216 [00:00<?, ?it/s]

precision_macro:  0.9116780570259306
recall_macro:  0.830116469186658
F1_macro_Score: :  0.8662364456052478
Accuracy:  0.991532219780816


## Testing in specific locations

In [ ]:
for i in range(0, len(list(configs_fs_global))):
  conf = configs_fs_global[list(configs_fs_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  wandb.run.name = "test_accuracy_feature_stack_tibet_thailand_global"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  test_1 = preproc.get_test_dataset("train_in_global/", "test_patches_1")
  test_2 = preproc.get_test_dataset("train_in_global/", "test_patches_2")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator(model_custom, test_1, 240)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator(model_custom, test_2, 240)
  df_test1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_test2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  wandb.log({'df_featurestack_test1_global': df_test1})
  wandb.log({'df_featurestack_test2_global': df_test2})

gs://geebucketwater/fs_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.979463225430326
recall_macro:  0.843463326213286
F1_macro_Score: :  0.8970405523035762
Accuracy:  0.9646963755289714


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.5567558638033063
recall_macro:  0.5000282153979797
F1_macro_Score: :  0.4408578682322836
Accuracy:  0.7880521774291992
gs://geebucketwater/fs_global_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


...

  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.464297192510022
recall_macro:  0.49999607755616654
F1_macro_Score: :  0.4703113608267296
Accuracy:  0.887887446085612


  0%|          | 0/240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision_macro:  0.394040584564209
recall_macro:  0.5
F1_macro_Score: :  0.44074127211605285
Accuracy:  0.788081169128418
gs://geebucketwater/fs_global_Cnn_L8SR_el_as/Models/L8SR_el_as_EPOCHS_10
L8SR_el_as


In [ ]:
for i in range(0, len(list(configs_fs_global))):
  print(i)
  conf = configs_fs_global[list(configs_fs_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  wandb.run.name = "Global_test_accuracy_feature_stack_tibet_thailand_10_Areas"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})

  test_1 = preproc.get_test_dataset("train_in_global/", "test_patches_g0")
  test_2 = preproc.get_test_dataset("train_in_global/", "test_patches_g1")
  test_3 = preproc.get_test_dataset("train_in_global/", "test_patches_g2")
  test_4 = preproc.get_test_dataset("train_in_global/", "test_patches_g3")
  test_5 = preproc.get_test_dataset("train_in_global/", "test_patches_g4")
  test_6 = preproc.get_test_dataset("train_in_global/", "test_patches_g5")
  test_7 = preproc.get_test_dataset("train_in_global/", "test_patches_g6")
  test_8 = preproc.get_test_dataset("train_in_global/", "test_patches_g7")
  test_9 = preproc.get_test_dataset("train_in_global/", "test_patches_g8")
  test_10 = preproc.get_test_dataset("train_in_global/", "test_patches_g9")
  
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator(model_custom, test_1, 72)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator(model_custom, test_2, 72)
  precision_t3, recall_t3, F1_t3, accuracy_t3 = metrics_.MetricCalculator(model_custom, test_3, 72)
  precision_t4, recall_t4, F1_t4, accuracy_t4 = metrics_.MetricCalculator(model_custom, test_4, 72)
  precision_t5, recall_t5, F1_t5, accuracy_t5 = metrics_.MetricCalculator(model_custom, test_5, 72)
  precision_t6, recall_t6, F1_t6, accuracy_t6 = metrics_.MetricCalculator(model_custom, test_6, 72)
  precision_t7, recall_t7, F1_t7, accuracy_t7 = metrics_.MetricCalculator(model_custom, test_7, 72)
  precision_t8, recall_t8, F1_t8, accuracy_t8 = metrics_.MetricCalculator(model_custom, test_8, 72)
  precision_t9, recall_t9, F1_t9, accuracy_t9 = metrics_.MetricCalculator(model_custom, test_9, 72)
  precision_t10, recall_t10, F1_t10, accuracy_t10 = metrics_.MetricCalculator(model_custom, test_10, 72)

  df_test1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_test2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  df_test3.loc[i] = [conf.PROJECT_TITLE + " test3"] + \
  [precision_t3] + [recall_t3] + [F1_t3] + [accuracy_t3]
  df_test4.loc[i] = [conf.PROJECT_TITLE + " test4"] + \
  [precision_t4] + [recall_t4] + [F1_t4] + [accuracy_t4]
  df_test5.loc[i] = [conf.PROJECT_TITLE + " test5"] + \
  [precision_t5] + [recall_t5] + [F1_t5] + [accuracy_t5]
  df_test6.loc[i] = [conf.PROJECT_TITLE + " test6"] + \
  [precision_t6] + [recall_t6] + [F1_t6] + [accuracy_t6]
  df_test7.loc[i] = [conf.PROJECT_TITLE + " test7"] + \
  [precision_t7] + [recall_t7] + [F1_t7] + [accuracy_t7]
  df_test8.loc[i] = [conf.PROJECT_TITLE + " test8"] + \
  [precision_t8] + [recall_t8] + [F1_t8] + [accuracy_t8]
  df_test9.loc[i] = [conf.PROJECT_TITLE + " test9"] + \
  [precision_t9] + [recall_t9] + [F1_t9] + [accuracy_t9]
  df_test10.loc[i] = [conf.PROJECT_TITLE + " test10"] + \
  [precision_t10] + [recall_t10] + [F1_t10] + [accuracy_t10]

  wandb.log({'df_global_featurestack_test1': df_test1})
  wandb.log({'df_global_featurestack_test2': df_test2})
  wandb.log({'df_global_featurestack_test3': df_test3})
  wandb.log({'df_global_featurestack_test4': df_test4})
  wandb.log({'df_global_featurestack_test5': df_test5})
  wandb.log({'df_global_featurestack_test6': df_test6})
  wandb.log({'df_global_featurestack_test7': df_test7})
  wandb.log({'df_global_featurestack_test8': df_test8})
  wandb.log({'df_global_featurestack_test9': df_test9})
  wandb.log({'df_global_featurestack_test10': df_test10})

0
gs://geebucketwater/fs_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/72 [00:00<?, ?it/s]

precision_macro:  0.975220057467679
recall_macro:  0.7975196604529668
F1_macro_Score: :  0.8606817174866197
Accuracy:  0.9557965596516927


  0%|          | 0/72 [00:00<?, ?it/s]

precision_macro:  0.7728974530507546
recall_macro:  0.5006583284781185
F1_macro_Score: :  0.4497797947594925
Accuracy:  0.8127782609727647


...

precision_macro:  0.9859450401346695
recall_macro:  0.9383944581892584
F1_macro_Score: :  0.9607991836570058
Accuracy:  0.9926160176595052


  0%|          | 0/72 [00:00<?, ?it/s]

precision_macro:  0.9870253133045384
recall_macro:  0.9460114952341522
F1_macro_Score: :  0.96555163205179
Accuracy:  0.9957667456732856


  0%|          | 0/72 [00:00<?, ?it/s]

precision_macro:  0.9447433508610992
recall_macro:  0.8822004186186484
F1_macro_Score: :  0.9109198578066753
Accuracy:  0.9888195461697049


## Test Everywhere

In [ ]:
# pat = 'gs://' + conf.BUCKET + '/' + "train_in_global/" + "test_patches" + '*'
# glob = tf.io.gfile.glob(pat)
# print(glob)

['gs://geebucketwater/train_in_global/test_patches_g0.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g1.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g2.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g3.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g4.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g5.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g6.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g7.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g8.tfrecord.gz', 'gs://geebucketwater/train_in_global/test_patches_g9.tfrecord.gz']


In [ ]:
for i in range(0, len(list(configs_fs_global))):
  print(i)
  conf = configs_fs_global[list(configs_fs_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "global_test_accuracy_feature_stack_everywhere_thailand"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})
  test = preproc.get_test_dataset("train_in_global/", "test_patches")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator(model_custom, test, 72*10)
  df_test_everywhere_1.loc[i] = [conf.PROJECT_TITLE] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  wandb.log({'global_df_featurestack_th_everywhere': df_test_everywhere_1})

0
gs://geebucketwater/fs_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9379093184029215
recall_macro:  0.6361433698997718
F1_macro_Score: :  0.697899465253199
Accuracy:  0.9483234193589952
1
gs://geebucketwater/fs_global_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.8824669568197565
recall_macro:  0.5503560448212782
F1_macro_Score: :  0.5745192783720193
Accuracy:  0.9364793353610569
2
gs://geebucketwater/fs_global_Cnn_S1A_sl/Models/S1A_sl_EPOCHS_10
S1A_sl


...

precision_macro:  0.6880351790883865
recall_macro:  0.5009534955191924
F1_macro_Score: :  0.48418152536326
Accuracy:  0.9308468076917861
30
gs://geebucketwater/fs_global_Cnn_L8SR_S1/Models/L8SR_S1_EPOCHS_10
L8SR_S1


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9341917481732849
recall_macro:  0.9413538430103985
F1_macro_Score: :  0.9377380051695687
Accuracy:  0.9838439729478624
31
gs://geebucketwater/fs_global_Cnn_L8SR_S1A/Models/L8SR_S1A_EPOCHS_10
L8SR_S1A


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.952785766239568
recall_macro:  0.8526436217995756
F1_macro_Score: :  0.8954615036648704
Accuracy:  0.9760436375935873


# Multiview-2

In [ ]:
for i in range(0, len(list(configs_multi))):
  print(i)
  conf = configs_multi[list(configs_multi)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "metrics_train_eval_multiview-2_tibet_thailand"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  training = preproc.get_training_dataset_for_testing("Train_in_Thailand/")
  evaluation = preproc.get_eval_dataset("Train_in_Thailand/")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_2(model_custom, training, 240*3)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator_multiview_2(model_custom, evaluation, 240*2)
  df_train_2.loc[i] = [conf.PROJECT_TITLE + " train"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_eval_2.loc[i] = [conf.PROJECT_TITLE + " test"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  wandb.log({'df_multiview-2_train': df_train_2})
  wandb.log({'df_multiview-2_eval': df_eval_2})

0
gs://geebucketwater/m2_TH_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9885726747997337
recall_macro:  0.9865665721636094
F1_macro_Score: :  0.9875669778447463
Accuracy:  0.9955517450968424


  0%|          | 0/480 [00:00<?, ?it/s]

precision_macro:  0.9914594539124671
recall_macro:  0.9144079947707909
F1_macro_Score: :  0.9493146173186315
Accuracy:  0.9915751775105794
1
gs://geebucketwater/m2_TH_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


...

  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.980288271720538
recall_macro:  0.9900315986341646
F1_macro_Score: :  0.9850969184505975
Accuracy:  0.9946027755737304


  0%|          | 0/480 [00:00<?, ?it/s]

precision_macro:  0.9860340639685616
recall_macro:  0.967287739085524
F1_macro_Score: :  0.9764563103338204
Accuracy:  0.9958370526631674
29
gs://geebucketwater/m2_TH_Cnn_L8SR_S1A_el/Models/L8SR_S1A_el_EPOCHS_10
L8SR_S1A_el


  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
for i in range(0, len(list(configs_multi_global))):
  print(i)
  conf = configs_multi_global[list(configs_multi_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "Global_test_accuracy_multiview_2_10_Areas"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})

  test_1 = preproc.get_test_dataset("train_in_global/", "test_patches_g0")
  test_2 = preproc.get_test_dataset("train_in_global/", "test_patches_g1")
  test_3 = preproc.get_test_dataset("train_in_global/", "test_patches_g2")
  test_4 = preproc.get_test_dataset("train_in_global/", "test_patches_g3")
  test_5 = preproc.get_test_dataset("train_in_global/", "test_patches_g4")
  test_6 = preproc.get_test_dataset("train_in_global/", "test_patches_g5")
  test_7 = preproc.get_test_dataset("train_in_global/", "test_patches_g6")
  test_8 = preproc.get_test_dataset("train_in_global/", "test_patches_g7")
  test_9 = preproc.get_test_dataset("train_in_global/", "test_patches_g8")
  test_10 = preproc.get_test_dataset("train_in_global/", "test_patches_g9")
  
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_2(model_custom, test_1, 72)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator_multiview_2(model_custom, test_2, 72)
  precision_t3, recall_t3, F1_t3, accuracy_t3 = metrics_.MetricCalculator_multiview_2(model_custom, test_3, 72)
  precision_t4, recall_t4, F1_t4, accuracy_t4 = metrics_.MetricCalculator_multiview_2(model_custom, test_4, 72)
  precision_t5, recall_t5, F1_t5, accuracy_t5 = metrics_.MetricCalculator_multiview_2(model_custom, test_5, 72)
  precision_t6, recall_t6, F1_t6, accuracy_t6 = metrics_.MetricCalculator_multiview_2(model_custom, test_6, 72)
  precision_t7, recall_t7, F1_t7, accuracy_t7 = metrics_.MetricCalculator_multiview_2(model_custom, test_7, 72)
  precision_t8, recall_t8, F1_t8, accuracy_t8 = metrics_.MetricCalculator_multiview_2(model_custom, test_8, 72)
  precision_t9, recall_t9, F1_t9, accuracy_t9 = metrics_.MetricCalculator_multiview_2(model_custom, test_9, 72)
  precision_t10, recall_t10, F1_t10, accuracy_t10 = metrics_.MetricCalculator_multiview_2(model_custom, test_10, 72)

  df_test1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_test2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  df_test3.loc[i] = [conf.PROJECT_TITLE + " test3"] + \
  [precision_t3] + [recall_t3] + [F1_t3] + [accuracy_t3]
  df_test4.loc[i] = [conf.PROJECT_TITLE + " test4"] + \
  [precision_t4] + [recall_t4] + [F1_t4] + [accuracy_t4]
  df_test5.loc[i] = [conf.PROJECT_TITLE + " test5"] + \
  [precision_t5] + [recall_t5] + [F1_t5] + [accuracy_t5]
  df_test6.loc[i] = [conf.PROJECT_TITLE + " test6"] + \
  [precision_t6] + [recall_t6] + [F1_t6] + [accuracy_t6]
  df_test7.loc[i] = [conf.PROJECT_TITLE + " test7"] + \
  [precision_t7] + [recall_t7] + [F1_t7] + [accuracy_t7]
  df_test8.loc[i] = [conf.PROJECT_TITLE + " test8"] + \
  [precision_t8] + [recall_t8] + [F1_t8] + [accuracy_t8]
  df_test9.loc[i] = [conf.PROJECT_TITLE + " test9"] + \
  [precision_t9] + [recall_t9] + [F1_t9] + [accuracy_t9]
  df_test10.loc[i] = [conf.PROJECT_TITLE + " test10"] + \
  [precision_t10] + [recall_t10] + [F1_t10] + [accuracy_t10]

  wandb.log({'df_global_multiview2_test1': df_test1})
  wandb.log({'df_global_multiview2_test2': df_test2})
  wandb.log({'df_global_multiview2_test3': df_test3})
  wandb.log({'df_global_multiview2_test4': df_test4})
  wandb.log({'df_global_multiview2_test5': df_test5})
  wandb.log({'df_global_multiview2_test6': df_test6})
  wandb.log({'df_global_multiview2_test7': df_test7})
  wandb.log({'df_global_multiview2_test8': df_test8})
  wandb.log({'df_global_multiview2_test9': df_test9})
  wandb.log({'df_global_multiview2_test10': df_test10})

0
gs://geebucketwater/m2_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


## Testing everywhere

In [ ]:
for i in range(0, len(list(configs_multi_global))):
  print(i)
  conf = configs_multi_global[list(configs_multi_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "global_test_accuracy_multi-2_everywhere"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})
  test = preproc.get_test_dataset("train_in_global/", "test_patches")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_2(model_custom, test, 72*10)
  df_test_everywhere_2.loc[i] = [conf.PROJECT_TITLE] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  wandb.log({'global_df_multi-2_everywhere': df_test_everywhere_2})

0
gs://geebucketwater/m2_global_Cnn_S1A_el_sl_as/Models/S1A_el_sl_as_EPOCHS_10
S1A_el_sl_as


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.7917854714805912
recall_macro:  0.7455612648558987
F1_macro_Score: :  0.7662448651745708
Accuracy:  0.9445739958021376
1
gs://geebucketwater/m2_global_Cnn_S1A_el/Models/S1A_el_EPOCHS_10
S1A_el


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.6132476346282436
recall_macro:  0.5487438379708709
F1_macro_Score: :  0.5631076282831475
Accuracy:  0.918862173292372
2
gs://geebucketwater/m2_global_Cnn_S1A_sl/Models/S1A_sl_EPOCHS_10
S1A_sl


...

precision_macro:  0.9742320451594544
recall_macro:  0.9462317797018118
F1_macro_Score: :  0.9597321156721719
Accuracy:  0.9899418089124892
35
gs://geebucketwater/m2_global_Cnn_L8SR_S1/Models/L8SR_S1_EPOCHS_10
L8SR_S1


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9659684606521874
recall_macro:  0.8637643717476888
F1_macro_Score: :  0.9075482121360499
Accuracy:  0.9787920633951823
36
gs://geebucketwater/m2_global_Cnn_L8SR_S1A/Models/L8SR_S1A_EPOCHS_10
L8SR_S1A


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9659253377433417
recall_macro:  0.9697488764107426
F1_macro_Score: :  0.9678278868970518
Accuracy:  0.9916861852010092


# Multiview-3

In [ ]:
for i in range(0, len(list(configs_multi_3))):
  print(i)
  conf = configs_multi_3[list(configs_multi_3)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "metrics_train_eval_multiview-3_tibet_thailand"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  training = preproc.get_training_dataset_for_testing("Train_in_Thailand/")
  evaluation = preproc.get_eval_dataset("Train_in_Thailand/")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_3(model_custom, training, 240*3)
  precision_t2, recall_t2, F1_t2, accuracy_t2 = metrics_.MetricCalculator_multiview_3(model_custom, evaluation, 240*2)
  df_train_2.loc[i] = [conf.PROJECT_TITLE + " train"] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  df_eval_2.loc[i] = [conf.PROJECT_TITLE + " test"] + \
  [precision_t2] + [recall_t2] + [F1_t2] + [accuracy_t2]
  wandb.log({'df_multiview-3_train': df_train_2})
  wandb.log({'df_multiview-3_eval': df_eval_2})

0
gs://geebucketwater/m3_TH_Cnn_L8SR_S1_as3/Models/L8SR_S1_as3_EPOCHS_10
L8SR_S1_as3


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9881309562499396
recall_macro:  0.9935732419709771
F1_macro_Score: :  0.9908327221724644
Accuracy:  0.996695073445638


  0%|          | 0/480 [00:00<?, ?it/s]

precision_macro:  0.9860239994309488
recall_macro:  0.9714159682172829
F1_macro_Score: :  0.9785962463126061
Accuracy:  0.9961990356445313
1
gs://geebucketwater/m3_TH_Cnn_L8SR_S1_el3/Models/L8SR_S1_el3_EPOCHS_10
L8SR_S1_el3


...

  0%|          | 0/480 [00:00<?, ?it/s]

precision_macro:  0.996396892033282
recall_macro:  0.9428619956517489
F1_macro_Score: :  0.9679421112248211
Accuracy:  0.994530995686849
5
gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_el3/Models/L8SR_S1A_el3_EPOCHS_10
L8SR_S1A_el3


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.975712926689956
recall_macro:  0.9913413919945104
F1_macro_Score: :  0.9833640619899258
Accuracy:  0.9939375983344184


  0%|          | 0/480 [00:00<?, ?it/s]

In [ ]:
for i in range(0, len(list(configs_multi_3))):
  print(i)
  conf = configs_multi_3[list(configs_multi_3)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  wandb.run.name = "test_accuracy_multiview-3_tibet_thailand"
  metrics_.CONFIG = conf
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})
  test_1 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_1")
  test_2 = preproc.get_test_dataset("Train_in_Thailand/", "test_patches_2")
  precision_t1, recall_t1, F1_mac_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_3(model_custom, test_1, 240)
  precision_t2, recall_t2, F1_mac_t2, accuracy_t2 = metrics_.MetricCalculator_multiview_3(model_custom, test_2, 240)
  df_test_m3_1.loc[i] = [conf.PROJECT_TITLE + " test1"] + \
  [precision_t1] + [recall_t1] + [F1_mac_t1] + [accuracy_t1]
  df_test_m3_2.loc[i] = [conf.PROJECT_TITLE + " test2"] + \
  [precision_t2] + [recall_t2] + [F1_mac_t2] + [accuracy_t2]
  wandb.log({'df_multiview-2_test1_': df_test_m3_1})
  wandb.log({'df_multiview-2_test2_': df_test_m3_2})

0
gs://geebucketwater/m3_TH_Cnn_L8SR_S1_as3/Models/L8SR_S1_as3_EPOCHS_10
L8SR_S1_as3


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.9837265431481339
recall_macro:  0.9875027659144204
F1_macro_Score: :  0.985604833264218
Accuracy:  0.9942462921142579


  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.7702388838126212
recall_macro:  0.6463483126889452
F1_macro_Score: :  0.6736463343828625
Accuracy:  0.8279109319051107
1
gs://geebucketwater/m3_TH_Cnn_L8SR_S1_el3/Models/L8SR_S1_el3_EPOCHS_10
L8SR_S1_el3


...

  0%|          | 0/240 [00:00<?, ?it/s]

precision_macro:  0.9804982853803367
recall_macro:  0.9712608461824526
F1_macro_Score: :  0.9758197771484869
Accuracy:  0.9904652277628581


  0%|          | 0/240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision_macro:  0.394040584564209
recall_macro:  0.5
F1_macro_Score: :  0.44074127211605285
Accuracy:  0.788081169128418


## Test Everywhere

In [ ]:
for i in range(5, len(list(configs_multi_3_global))):
  print(i)
  conf = configs_multi_3_global[list(configs_multi_3_global)[i]]
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  print(MODEL_DIR)
  print(conf.PROJECT_TITLE)
  metrics_.CONFIG = conf
  wandb.run.name = "global_test_accuracy_multi-3_everywhere"
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})
  test = preproc.get_test_dataset("train_in_global/", "test_patches")
  precision_t1, recall_t1, F1_t1, accuracy_t1 = metrics_.MetricCalculator_multiview_3(model_custom, test, 72*10)
  df_test_everywhere_3.loc[i] = [conf.PROJECT_TITLE] + \
  [precision_t1] + [recall_t1] + [F1_t1] + [accuracy_t1]
  wandb.log({'global_df_multi-3_everywhere': df_test_everywhere_3})

5
gs://geebucketwater/m3_global_Cnn_L8SR_S1A_el3/Models/L8SR_S1A_el3_EPOCHS_10
L8SR_S1A_el3


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9627164475392491
recall_macro:  0.9676771543513196
F1_macro_Score: :  0.9651811817100838
Accuracy:  0.990990956624349
6
gs://geebucketwater/m3_global_Cnn_L8SR_S1A_sl3/Models/L8SR_S1A_sl3_EPOCHS_10
L8SR_S1A_sl3


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.9693060770672783
recall_macro:  0.9717873887262518
F1_macro_Score: :  0.9705428751265122
Accuracy:  0.9923988554212783
7
gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Models/L8SR_S1A_sl_el_as3_EPOCHS_10
L8SR_S1A_sl_el_as3


  0%|          | 0/720 [00:00<?, ?it/s]

precision_macro:  0.970584159925806
recall_macro:  0.968270019620592
F1_macro_Score: :  0.969423729179386
Accuracy:  0.9921502219306098
